<a href="https://colab.research.google.com/github/goelpravin/practice/blob/master/segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/tensorflow/examples.git
!pip install -U tfds-nightly

  Cloning https://github.com/tensorflow/examples.git to /tmp/pip-req-build-2rx8t4zv
  Running command git clone -q https://github.com/tensorflow/examples.git /tmp/pip-req-build-2rx8t4zv
  Created wheel for tensorflow-examples: filename=tensorflow_examples-fa50f88293ef253803c7fcf1ea9c562f27e58656_-cp36-none-any.whl size=135402 sha256=15861f82572003145a25209e90ff8522576b58f99d1db75d6172f0cee8abc07a
  Stored in directory: /tmp/pip-ephem-wheel-cache-f3c1zvxc/wheels/83/64/b3/4cfa02dc6f9d16bf7257892c6a7ec602cd7e0ff6ec4d7d714d
Successfully built tensorflow-examples
     |████████████████████████████████| 3.5MB 4.5MB/s 


In [2]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()
dataset, info = tfds.load('oxford_iiit_pet:3.*.*', with_info=True)

Shuffling and writing examples to /root/tensorflow_datasets/oxford_iiit_pet/3.2.0.incompleteNSJF90/oxford_iiit_pet-train.tfrecord
Shuffling and writing examples to /root/tensorflow_datasets/oxford_iiit_pet/3.2.0.incompleteNSJF90/oxford_iiit_pet-test.tfrecord
Dataset oxford_iiit_pet downloaded and prepared to /root/tensorflow_datasets/oxford_iiit_pet/3.2.0. Subsequent calls will reuse this data.


In [3]:
import tensorflow as tf


def getImageMask(datapoint):
  image = tf.image.resize(datapoint['image'], (128, 128))
  mask = tf.image.resize(datapoint['segmentation_mask'], (128, 128))
  return tf.expand_dims(image, axis=0), tf.expand_dims(mask, axis=0)



In [62]:
from keras_preprocessing.image import NumpyArrayIterator
def tfds_imgen_train(ds, imgen,maskgen, batch_size):
  # Provide the same seed and keyword arguments to the fit and flow methods
  seed = 1
  
  for images, segmentation_masks in ds:
    #print("labels in tfdsimgen train",segmentation_masks)
    imgen.fit(images, augment=True, seed=seed)
    maskgen.fit(segmentation_masks, augment=True, seed=seed)
    image_generator = imgen.flow(
        x=images,
        seed=seed)
    mask_generator = maskgen.flow(
        x=segmentation_masks,
        seed=seed)
    train_generator = zip(image_generator, mask_generator)
    return train_generator
    print("image",image_generator.shape)
    print("mask",mask_generator.shape)
    #yield image_generator, mask_generator
    #return (NumpyArrayIterator(images,segmentation_masks,imgen,batch_size=batch_size))
    #return_value = next( imgen.flow(x=images, y=labels, batch_size=batch_size))
    #print(return_value)
    #yield return_value
def tfds_imgen_validate(ds, imgen, maskgen,batch_size):
  # Provide the same seed and keyword arguments to the fit and flow methods
  seed = 2
  
  for images, labels in ds:
    print("labels in tfdsimgen validate",labels)
    imgen.fit(images, augment=True, seed=seed)
    maskgen.fit(labels, augment=True, seed=seed)
    image_generator = next(imgen.flow(
        x=images,
        seed=seed))
    mask_generator = next(maskgen.flow(
        x=labels,
        seed=seed))
    #zipped_generator = zip(image_generator, mask_generator)
    #yield image_generator, mask_generator
    return (NumpyArrayIterator(images,labels,imgen))
    #return_value = next( imgen.flow(x=images, y=labels, batch_size=batch_size))
    #print(return_value)
    #yield return_value


In [63]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
train_ds = dataset['train'].map(getImageMask)
test_ds = dataset['test'].map(getImageMask)

train_datagen = ImageDataGenerator(
      rescale=1/255)#,
      #rotation_range=40,
      #width_shift_range=0.2,
      #height_shift_range=0.2,
      #shear_range=0.2,
      #zoom_range=0.2,
      #horizontal_flip=True,
      #fill_mode='nearest')
# we create two instances with the same arguments
train_data_gen_args = dict(featurewise_center=True,
                     featurewise_std_normalization=True,
                     rotation_range=40,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0.2)
train_image_datagen = ImageDataGenerator(**train_data_gen_args)
train_mask_datagen = ImageDataGenerator(**train_data_gen_args)

validation_image_datagen = ImageDataGenerator(rescale=1/255)
validation_mask_datagen = ImageDataGenerator(rescale=1/255)
batch_size_training = 128
batch_size_validation = 32
train_generator = tfds_imgen_train(train_ds.as_numpy_iterator(),train_image_datagen,train_mask_datagen,batch_size_training)
validation_generator = tfds_imgen_validate(test_ds.as_numpy_iterator(),validation_image_datagen,validation_mask_datagen,batch_size_validation)


labels in tfdsimgen validate [[[[2.]
   [2.]
   [2.]
   ...
   [2.]
   [2.]
   [2.]]

  [[2.]
   [2.]
   [2.]
   ...
   [2.]
   [2.]
   [2.]]

  [[2.]
   [2.]
   [2.]
   ...
   [2.]
   [2.]
   [2.]]

  ...

  [[2.]
   [2.]
   [2.]
   ...
   [2.]
   [2.]
   [2.]]

  [[2.]
   [2.]
   [2.]
   ...
   [2.]
   [2.]
   [2.]]

  [[2.]
   [2.]
   [2.]
   ...
   [2.]
   [2.]
   [2.]]]]


In [17]:
def show_predictions(dataset=None, num=1):
  if dataset:
    for image, mask in dataset.take(num):
      pred_mask = model.predict(image)
      #display([tf.squeeze(image[0],axis=0), tf.squeeze(mask[0],axis=0), create_mask(pred_mask)])
      display(image, mask, create_mask(pred_mask))
  else:
    for image, mask in train_ds.take(1):
      img2predict, mask2predict = image, mask
    display([sample_image, sample_mask,
             create_mask(model.predict(img2predict))])
             #create_mask(model.predict(sample_image))])

In [18]:
def create_mask(pred_mask):
  print("pred mask",pred_mask)
  pred_mask = tf.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0]

In [19]:
def display(display_list):
  plt.figure(figsize=(15, 15))

  title = ['Input Image', 'True Mask', 'Predicted Mask']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    #print(display_list[i])

    plt.title(title[i])
    plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
    plt.axis('off')
  plt.show()
for image, mask in train_ds.take(1):
  sample_image, sample_mask = tf.squeeze(image,axis=0), tf.squeeze(mask,axis=0)
#display([sample_image, sample_mask])
#show_predictions()

In [20]:
from tensorflow_examples.models.pix2pix import pix2pix
from tensorflow.keras.optimizers import Adam
OUTPUT_CHANNELS = 3
base_model = tf.keras.applications.MobileNetV2(input_shape=[128, 128, 3], include_top=False)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
def compute_loss(labels, predictions):
  print("labels=",labels)
  print("predictions=",predictions)
  per_example_loss = loss_object(labels, predictions)
  #per_example_loss /= tf.cast(
   #   tf.shape(labels)[1] * tf.shape(labels)[2], tf.float32)  # <--- See if this fixes the NaNs.
  return tf.nn.compute_average_loss(per_example_loss)
      #per_example_loss, global_batch_size=GLOBAL_BATCH_SIZE)
# Use the activations of these layers
layer_names = [
    'block_1_expand_relu',   # 64x64
    'block_3_expand_relu',   # 32x32
    'block_6_expand_relu',   # 16x16
    'block_13_expand_relu',  # 8x8
    'block_16_project',      # 4x4
]
layers = [base_model.get_layer(name).output for name in layer_names]

# Create the feature extraction model
down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)

down_stack.trainable = False


up_stack = [
    pix2pix.upsample(512, 3),  # 4x4 -> 8x8
    pix2pix.upsample(256, 3),  # 8x8 -> 16x16
    pix2pix.upsample(128, 3),  # 16x16 -> 32x32
    pix2pix.upsample(64, 3),   # 32x32 -> 64x64
]

def unet_model(output_channels):
  inputs = tf.keras.layers.Input(shape=[128, 128, 3])
  x = inputs

  # Downsampling through the model
  skips = down_stack(x)
  x = skips[-1]
  skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    concat = tf.keras.layers.Concatenate()
    x = concat([x, skip])

  # This is the last layer of the model
  last = tf.keras.layers.Conv2DTranspose(
      output_channels, 3, strides=2,
      padding='same')  #64x64 -> 128x128

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

model = unet_model(OUTPUT_CHANNELS)
print("test")
model.compile(optimizer=Adam(lr=0.001),
              #loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              loss=compute_loss,
              metrics=['accuracy'])


test


In [64]:
from IPython.display import clear_output
class myCallback (tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    clear_output(wait=True)
    show_predictions()
    if(logs.get('accuracy')>0.8):
      print("\nReached 80% accuracy, so stopping further epochs")
      self.model.stop_training = True
callbacks = myCallback()


num_training_images = info.splits['train'].num_examples
epoch_steps = num_training_images/batch_size_training

num_testing_images = info.splits['test'].num_examples
validation_steps = num_testing_images/batch_size_validation

history = model.fit(
      train_generator,
      steps_per_epoch=epoch_steps,  
      epochs=10,
      verbose=2,
      validation_data = validation_generator,
      validation_steps=validation_steps,
      callbacks=[callbacks])

Epoch 1/10
labels= Tensor("IteratorGetNext:1", shape=(None, None, None, None), dtype=float32)
predictions= Tensor("functional_7/conv2d_transpose_9/BiasAdd:0", shape=(None, 128, 128, 3), dtype=float32)


ValueError: ignored

In [ ]:
show_predictions(test_ds)

pred mask [[[[nan nan nan]
   [nan nan nan]
   [nan nan nan]
   ...
   [nan nan nan]
   [nan nan nan]
   [nan nan nan]]

  [[nan nan nan]
   [nan nan nan]
   [nan nan nan]
   ...
   [nan nan nan]
   [nan nan nan]
   [nan nan nan]]

  [[nan nan nan]
   [nan nan nan]
   [nan nan nan]
   ...
   [nan nan nan]
   [nan nan nan]
   [nan nan nan]]

  ...

  [[nan nan nan]
   [nan nan nan]
   [nan nan nan]
   ...
   [nan nan nan]
   [nan nan nan]
   [nan nan nan]]

  [[nan nan nan]
   [nan nan nan]
   [nan nan nan]
   ...
   [nan nan nan]
   [nan nan nan]
   [nan nan nan]]

  [[nan nan nan]
   [nan nan nan]
   [nan nan nan]
   ...
   [nan nan nan]
   [nan nan nan]
   [nan nan nan]]]]


TypeError: ignored